In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import glob
from time import sleep
from tqdm import tqdm
import cv2

In [ ]:
image = glob.glob(r'../input/lungdatasets3/lung2/label/train_label/*.png')
counts=len(image)
i = 1
j = counts
img = []
label=[]
while i <=j:
    img_path=str('../input/lungdatasets3/lung2/image/train_image/'+str(i)+'.jpg')
    i+=1
    img.append(img_path)
img= [str(path) for path in img]
i =1
j = counts
while i <=j:
    k=0
    label_path=str('../input/lungdatasets3/lung2/label/train_label/'+str(i)+'.png')
    i+=1
    label.append(label_path)
    k+=1
label= [str(path) for path in label]
val=glob.glob(r'../input/lungdatasets3/lung2/label/test_label/*.png')
counts=len(val)
i = 1
j =counts
img_val = []
label_val=[]
while i <j:
    img_path=str('../input/lungdatasets3/lung2/image/test_image/'+str(i)+'.jpg')
    i+=1
    img_val.append(img_path)
img_val= [str(path) for path in img_val]
i =1
j =counts
while i <j:
    k=0
    label_path=str('../input/lungdatasets3/lung2/label/test_label/'+str(i)+'.png')
    i+=1
    label_val.append(label_path)
    k+=1
label_val= [str(path) for path in label_val]
train_count =len(img)
val_count = len(img_val)
dataset_train = tf.data.Dataset.from_tensor_slices((img, label))
dataset_val = tf.data.Dataset.from_tensor_slices((img_val, label_val))
def read_jpg(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    return img
def read_png_label(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=1)
    return img
def normal(img, mask):
    img = tf.cast(img, tf.float32)/127.5 -1
    mask = mask
    mask = tf.cast(mask, tf.int32)
    return img, mask
                                                                                                                                                                                                                                             

    if tf.random.uniform(()) > 0.5:
        img = tf.image.flip_left_right(img)
        mask = tf.image.flip_left_right(mask)

    img, mask = normal(img, mask)

    return img, mask



dataset_train = dataset_train.map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset_val = dataset_val.map(load_image_val, num_parallel_calls=tf.data.experimental.AUTOTUNE)

BATCH_SIZE =4
dataset_train =dataset_train.repeat(count=1).shuffle(50).batch(BATCH_SIZE)

dataset_val =dataset_val.batch(1)

for i, m in dataset_train.take(50):
    plt.subplot(1, 2, 1)
    plt.imshow(tf.keras.preprocessing.image.array_to_img(i[0]))
    plt.subplot(1, 2, 2)           
    plt.imshow(tf.keras.preprocessing.image.array_to_img(m[0]))



In [ ]:
for i, m in dataset_train.take(50):
    plt.subplot(1, 2, 1)
    plt.imshow(tf.keras.preprocessing.image.array_to_img(i[0]))
    plt.subplot(1, 2, 2)           
    plt.imshow(tf.keras.preprocessing.image.array_to_img(m[0]))



In [ ]:
class SplitConv2(keras.layers.Layer):
    r = 5
    reduction_factor = 2
    def __init__(self,units, kernel_size=3, strides=1,padding='same'):
        super(SplitConv2, self).__init__()
        self.conv = Conv2(units, kernel_size=3, strides=strides,padding='same')
        self.conv4 = Conv2(units, kernel_size=3, strides=strides,padding='same')
        self.conv5 = Conv2(units, kernel_size=3, strides=strides,padding='same')
        self.conv6 = Conv2(units, kernel_size=3, strides=strides,padding='same')
        self.conv7 = Conv2(units, kernel_size=3, strides=strides,padding='same')
        self.pool = keras.layers.GlobalAveragePooling2D()
        self.bn= keras.layers.BatchNormalization(momentum =0.9 ,epsilon =1e-5)
        self.conv1 = keras.layers.Conv2D(units*3*3, kernel_size=1, padding='same')
        self.conv2 = keras.layers.Conv2D(units*3*3, kernel_size=1, padding='same')
        self.conv3 = keras.layers.Conv2D(units*3, kernel_size=1, padding='same')
        self.conv8 = keras.layers.Conv2D(units*3//2, kernel_size=1, padding='same')
        self.conv9= keras.layers.Conv2D(units*3, kernel_size=1, padding='same')
        self.sigmoid = tf.keras.layers.Activation('softmax')
        self.n = units
        self.m = units
        self.strides=strides
        self.spatial =keras.layers.Conv2D(1,kernel_size = 7,strides=1,padding='same')
    def call(self,x):
        n = self.n
        m = self.m*3
        strides =self.strides

        k=tf.split(x,2,axis=-1)
        k0=self.conv3(k[0])
            
        x1=tf.split(k0,3,axis=-1)
        x10 = self.conv(x1[0])
        x11 = self.conv4(x1[1])
        x12 = self.conv5(x1[2])
        x_origin = x10
        x1 = tf.add_n([x10,x11,x12])
        x1 = self.pool(x1)
        x1 = tf.expand_dims(x1,axis=1)
        x1 = tf.expand_dims(x1,axis=1)
        if strides ==2:
            x1 = self.conv8(x1)
        else:
            x1 = self.conv1(x1)
        x1 = self.bn(x1)
        x1 = tf.nn.relu(x1)
        if strides ==2:
            m=self.m
            x1 = self.conv9(x1)
        else:
            x1 = self.conv2(x1)
        x1 = self.bn(x1)
        x1 = tf.nn.relu(x1)
        x1 = self.sigmoid(x1)
        x1= tf.keras.layers.UpSampling2D(size=(x_origin.shape[1],x_origin.shape[2]),interpolation = 'nearest')(x1)
        s=tf.split(x1,3,axis=-1)
        x_10=tf.multiply(s[0],x10)
        x_11=tf.multiply(s[1],x11)
        x_12=tf.multiply(s[2],x12)
        #x_3=tf.multiply(s[3],x3)
       # x_4=tf.multiply(s[4],x4)
        avgpool10 = tf.reduce_mean(x_10,axis=3,keepdims=True )
        maxpool10 = tf.reduce_max(x_10,axis=3,keepdims=True )
        spatial10 = tf.concat([avgpool10,maxpool10],axis=3)
        spatial10 = self.spatial(spatial10)
        spatial10 =self.sigmoid(spatial10)
        spatial11 = self.spatial(spatial11)
        spatial11 =self.sigmoid(spatial11)
        x_11  =tf.multiply(x_11,spatial11)
        avgpool12 = tf.reduce_mean(x_12,axis=3,keepdims=True )
        maxpool12 = tf.reduce_max(x_12,axis=3,keepdims=True )
        spatial12 = tf.concat([avgpool12,maxpool12],axis=3)
        spatial12 = self.spatial(spatial12)
        spatial12 =self.sigmoid(spatial12)
        x_12  =tf.multiply(x_12,spatial12)
        x1=tf.add_n([x_10,x_11,x_12])


        #x3 = self.conv6(x[3])
        
        k1=self.conv3(k[1])
        
        x2=tf.split(k1,3,axis=-1)
        x20 = self.conv(x2[0])
        x21 = self.conv4(x2[1])
        x22 = self.conv5(x2[2])
        x_origin = x20
        x2 = tf.add_n([x20,x21,x22])
        x2 = self.pool(x2)
        x2 = tf.expand_dims(x2,axis=1)
        x2 = tf.expand_dims(x2,axis=1)
        if strides ==2:
            x2 = self.conv8(x2)
        else:
            x2 = self.conv1(x2)
        x2 = self.bn(x2)
        x2 = tf.nn.relu(x2)
        if strides ==2:
            m=self.m
            x2 = self.conv9(x2)
        else:
            x2 = self.conv2(x2)
        x2 = self.bn(x2)
        x2 = tf.nn.relu(x2)
        x2 = self.sigmoid(x2)
        x2= tf.keras.layers.UpSampling2D(size=(x_origin.shape[1],x_origin.shape[2]),interpolation = 'nearest')(x2)
        s2=tf.split(x2,3,axis=-1)
        x_20=tf.multiply(s2[0],x20)
        x_21=tf.multiply(s2[1],x21)
        x_22=tf.multiply(s2[2],x22)
        #x_3=tf.multiply(s[3],x3)
       # x_4=tf.multiply(s[4],x4)
        avgpool20 = tf.reduce_mean(x_20,axis=3,keepdims=True )
        maxpool20 = tf.reduce_max(x_20,axis=3,keepdims=True )
        spatial20 = tf.concat([avgpool20,maxpool20],axis=3)
        spatial20 = self.spatial(spatial20)
        spatial20 =self.sigmoid(spatial20)
        x_20  =tf.multiply(x_20,spatial20)
        avgpool21 = tf.reduce_mean(x_21,axis=3,keepdims=True )
        maxpool21 = tf.reduce_max(x_21,axis=3,keepdims=True )
        spatial21 = tf.concat([avgpool21,maxpool21],axis=3)
        spatial21 = self.spatial(spatial21)
        spatial21 =self.sigmoid(spatial21)
        x_21  =tf.multiply(x_21,spatial21)
        avgpool22 = tf.reduce_mean(x_22,axis=3,keepdims=True )
        maxpool22 = tf.reduce_max(x_22,axis=3,keepdims=True )
        spatial22 = tf.concat([avgpool22,maxpool22],axis=3)
        spatial22 = self.spatial(spatial22)
        spatial22 =self.sigmoid(spatial22)
        x_22  =tf.multiply(x_22,spatial22)
       # avgpool3 = tf.reduce_mean(x_3,axis=3,keepdims=True )
        #maxpool3 = tf.reduce_max(x_3,axis=3,keepdims=True )
        #spatial3 = tf.concat([avgpool3,maxpool3],axis=3)
        #spatial3 = self.spatial(spatial3)
        #spatial3 =self.sigmoid(spatial3)
        #x_3  =tf.multiply(x_3,spatial3)
        #avgpool4 = tf.reduce_mean(x_4,axis=3,keepdims=True )
        #maxpool4 = tf.reduce_max(x_4,axis=3,keepdims=True )
        #spatial4 = tf.concat([avgpool4,maxpool4],axis=3)
        #spatial4 = self.spatial(spatial4)
        #spatial4 =self.sigmoid(spatial4)
        #x_4  =tf.multiply(x_4,spatial4)
        
        x2=tf.add_n([x_20,x_21,x_22])
        x =tf.concat([x2,x1],axis=-1)
        return x 

In [ ]:
class BasicBlock(keras.layers.Layer):
    def __init__(self,units,strides=1):
        super(BasicBlock, self).__init__()
        self.conv1 = SplitConv2(units, kernel_size=3, strides=strides,padding='same') 
        self.conv2 = keras.layers.Conv2D(units*2, kernel_size=1, padding='same') 
        self.bn2 = keras.layers.BatchNormalization(momentum =0.9 ,epsilon =1e-5)
        self.downsample = keras.layers.Conv2D(units*2, kernel_size=1,strides=1, padding='same')
        self.pool=tf.keras.layers.AveragePooling2D(pool_size=(2,2),strides=strides,padding='same')
    def call(self,x,is_downsample=False):
        identity = x
        if is_downsample:
            identity =self.pool(x)
            identity = self.downsample(identity)
        x = self.conv1(x)
         
        x = self.conv2(x)
        x = self.bn2(x)
        
        x = tf.add(x,identity)
        x = tf.nn.relu(x)
        return x

In [ ]:
class Downsample(keras.layers.Layer):
    def __init__(self, units):
        super(Downsample, self).__init__()
        self.conv1 = keras.layers.Conv2D(units, kernel_size=3, padding='same')
        self.conv2 = keras.layers.Conv2D(units, kernel_size=3, padding='same')
        self.pool = keras.layers.MaxPooling2D()
        
    def call(self, x, is_pool=True):
        if is_pool:
            x = self.pool(x)
        x = self.conv1(x)
        x = tf.nn.relu(x)
        x = self.conv2(x)
        x = tf.nn.relu(x)
        return x

In [ ]:
class Upsample(keras.layers.Layer):
    def __init__(self, unit):
        super(Upsample, self).__init__()
        self.conv1 = keras.layers.Conv2D(unit, kernel_size=3, padding='same')
        self.conv2 = keras.layers.Conv2D(unit, kernel_size=3, padding='same')
        self.deconv = keras.layers.Conv2DTranspose(unit//2,
                                                   kernel_size=3,
                                                   strides=2,
                                                   padding='same')
    def call(self, x, is_pool=True):
        x = self.conv1(x)
        x = tf.nn.relu(x)
        x = self.conv2(x)
        x = tf.nn.relu(x)
        x = self.deconv(x)
        x = tf.nn.relu(x)
        return x

In [ ]:
class Net(keras.Model):
    def __init__(self):
        super(Net, self).__init__()
        self.input_conv1 = keras.layers.Conv2D(64, kernel_size=3, strides=1,padding='same')
        self.input_conv2 = keras.layers.Conv2D(64, kernel_size=3, strides=1,padding='same')
        
        self.down1_1 = BasicBlock(64,strides = 2)
        self.down1_2 = BasicBlock(64,strides = 1)
        self.down1_3 = BasicBlock(64,strides = 1)
        
        self.down2_1 = BasicBlock(128,strides = 2)
        self.down2_2 = BasicBlock(128,strides = 1)
        self.down2_3 = BasicBlock(128,strides = 1)
        self.down2_4 = BasicBlock(128,strides = 1) 
        
        self.down3_1 = BasicBlock(256,strides = 2)
        self.down3_2 = BasicBlock(256,strides = 1)
        self.down3_3 = BasicBlock(256,strides = 1)
        self.down3_4 = BasicBlock(256,strides = 1)
        self.down3_5 = BasicBlock(256,strides = 1)
        self.down3_6 = BasicBlock(256,strides = 1)
        
        self.down4_1 = BasicBlock(512,strides = 2)
        self.down4_2 = BasicBlock(512,strides = 1)
        self.down4_3 = BasicBlock(512,strides = 1)
        
        
    
        #self.conv = tf.keras.layers.Conv2D(512,kernel_size=3,strides = 1,padding = 'same')
       
        self.up1 = Upsample(512)
        self.up2 = Upsample(256)
        self.up3 = Upsample(128)
     
        
        
        self.up = keras.layers.Conv2DTranspose(512,
                                               kernel_size=3,
                                               strides = 2,
                                               padding='same')
        self.conv_last = Downsample(64)
        
        
        self.last = keras.layers.Conv2D(2,kernel_size=1,strides =1,padding='same')
        
        self.convdown1_2 =keras.layers.Conv2D(128,kernel_size=1,strides=2,padding='same')
        self.convdown1_3 =keras.layers.Conv2D(256,kernel_size=1,strides=4,padding='same')
        self.convdown1_4 =keras.layers.Conv2D(512,kernel_size=1,strides=8,padding='same')
        
        self.convup2_1  =keras.layers.Conv2DTranspose(64,kernel_size=3,strides=2,padding='same')
        self.convdown2_3 =keras.layers.Conv2D(256,kernel_size=1,strides=2,padding='same')
        self.convdown2_4 =keras.layers.Conv2D(512,kernel_size=1,strides=4,padding='same')
        
        self.convup3_1  =keras.layers.Conv2DTranspose(64,kernel_size=3,strides=4,padding='same')
        self.convup3_2  =keras.layers.Conv2DTranspose(128,kernel_size=3,strides=2,padding='same')
        self.convdown3_4 =keras.layers.Conv2D(512,kernel_size=1,strides=2,padding='same')
        
        self.convup4_1  =keras.layers.Conv2DTranspose(64,kernel_size=3,strides=8,padding='same')
        self.convup4_2  =keras.layers.Conv2DTranspose(128,kernel_size=3,strides=4,padding='same')
        self.convup4_3  =keras.layers.Conv2DTranspose(256,kernel_size=3,strides=2,padding='same')
        
  
        self.convcat1 = keras.layers.Conv2D(64,kernel_size=1,strides=1,padding='same')
        self.convcat1_2 = keras.layers.Conv2D(64,kernel_size=1,strides=1,padding='same')
        self.bn1_1 = keras.layers.BatchNormalization(momentum =0.9 ,epsilon =1e-5)
        self.bn1_2 = keras.layers.BatchNormalization(momentum =0.9 ,epsilon =1e-5)
        
        self.convcat2 = keras.layers.Conv2D(128,kernel_size=1,strides=1,padding='same')
        self.convcat2_2 = keras.layers.Conv2D(128,kernel_size=1,strides=1,padding='same')
        self.bn2_1 = keras.layers.BatchNormalization(momentum =0.9 ,epsilon =1e-5)
        self.bn2_2= keras.layers.BatchNormalization(momentum =0.9 ,epsilon =1e-5)
        
        self.convcat3 = keras.layers.Conv2D(256,kernel_size=1,strides=1,padding='same')
        self.convcat3_2 = keras.layers.Conv2D(256,kernel_size=1,strides=1,padding='same')
        self.bn3_1 = keras.layers.BatchNormalization(momentum =0.9 ,epsilon =1e-5)
        self.bn3_2 = keras.layers.BatchNormalization(momentum =0.9 ,epsilon =1e-5)
        
        self.convcat4 = keras.layers.Conv2D(512,kernel_size=1,strides=1,padding='same')
        self.convcat4_2 = keras.layers.Conv2D(512,kernel_size=1,strides=1,padding='same')
        self.bn4_1 = keras.layers.BatchNormalization(momentum =0.9 ,epsilon =1e-5)
        self.bn4_2 = keras.layers.BatchNormalization(momentum =0.9 ,epsilon =1e-5)
        
        
        self.enhance = Enhance(units=256,padding='same')
        self.conv_e = tf.keras.layers.Conv2D(1024,kernel_size=1,strides=1,padding='same')
        self.bn1= keras.layers.BatchNormalization(momentum =0.9 ,epsilon =1e-5)

    def call(self,x):
        x1 =self.input_conv1(x)  #256*256*64
        x1 =self.input_conv2(x1)  #256*256*64
        
        
        
        
        x2 = self.down1_1(x1,is_downsample = True)  #128*128*128
        x2 = self.down1_2(x2)
        x2 = self.down1_3(x2)
       
        
        
        x3 = self.down2_1(x2,is_downsample = True)  #64*64*256
        x3 = self.down2_2(x3)
        x3 = self.down2_3(x3)
        x3 = self.down2_4(x3)
        
       
        
        x4 = self.down3_1(x3,is_downsample = True) #32*32*512
        x4 = self.down3_2(x4)
        x4 = self.down3_3(x4)
        x4 = self.down3_4(x4)
        x4 = self.down3_5(x4)
        x4 = self.down3_6(x4)

       
        x5 = self.down4_1(x4,is_downsample = True) #16*16*1024
        x5 = self.down4_2(x5)
        x5 = self.down4_3(x5)
            
        x5_e = self.enhance(x5)  
                                              
        #x5_4 =self.convdown5_4(x5_e) 
        #x5_3 =self.convdown5_3(x5_e)                                       
        #x5_2 =self.convdown5_2(x5_e)
        #x5_1 =self.convdown5_1(x5_e)                                     
                                              
        x5 = tf.concat([x5,x5_e],axis = -1)
        x5 = tf.nn.relu(x5)
        x5 = self.conv_e(x5)
        x5 = self.bn1(x5)
        x5 =tf.nn.relu(x5)
        
        x5 = self.up(x5)   
                
        x1_4 =self.convdown1_4(x1)            
        x2_4 =self.convdown2_4(x2)
        x3_4 =self.convdown3_4(x3)
        x4_s =tf.concat([x1_4,x2_4,x3_4],axis=-1)
        x4_s =tf.nn.relu(x4_s)
        x4_s =self.convcat4(x4_s)
        x4_s =self.bn4_1(x4_s)
        x4 =tf.concat([x4_s,x4],axis=-1)
        x4 =self.convcat4_2(x4)
        x4 = self.bn4_2(x4)
        x5 = tf.concat([x4,x5], axis=-1)      #32*32*1024
        x5 =tf.nn.relu(x5)
        
        x5 = self.up1(x5)     #32*32*512
        
        x1_3 =self.convdown1_3(x1)            
        x2_3 =self.convdown2_3(x2)
        x4_3 =self.convup4_3(x4)
        x3_s =tf.concat([x1_3,x2_3,x4_3],axis=-1)
        x3_s =tf.nn.relu(x3_s)
        x3_s =self.convcat3(x3_s)
        x3_s =self.bn3_1(x3_s)
        x3 =tf.concat([x3_s,x3],axis=-1)
        x3 =self.convcat3_2(x3)
        x3 =self.bn3_2(x3)
        x5 = tf.concat([x3,x5], axis=-1)      #64*64*512
        x5 =tf.nn.relu(x5)
        
        x5 = self.up2(x5)                     #128*128*128
        
        x1_2 =self.convdown1_2(x1)            
        x3_2 =self.convup3_2(x3)
        x4_2 =self.convup4_2(x4)
        x2_s =tf.concat([x1_2,x3_2,x4_2],axis=-1)
        x2_s =tf.nn.relu(x2_s)
        x2_s =self.convcat2(x2_s)
        x2_s =self.bn2_1(x2_s)
        x2 =tf.concat([x2_s,x2],axis=-1)
        x2 =self.convcat2_2(x2)
        x2 =self.bn2_2(x2)
        x5 = tf.concat([x2,x5], axis=-1)      #128*128*256
        x5 =tf.nn.relu(x5)
        
        x5 = self.up3(x5)                     #256*256*64
        
        x2_1 =self.convup2_1(x2)            
        x3_1 =self.convup3_1(x3)
        x4_1 =self.convup4_1(x4)
        x1_s =tf.concat([x2_1,x3_1,x4_1],axis=-1)
        x1_s =tf.nn.relu(x1_s)
        x1_s =self.convcat1(x1_s)
        x1_s =self.bn1_1(x1_s)
        x1 =tf.concat([x1_s,x1],axis=-1)
        x1 =self.convcat1_2(x1)
        x1 =self.bn1_2(x1)
        x5 = tf.concat([x1,x5], axis=-1)      #256*256*128
        x5 =tf.nn.relu(x5)
        
          
        x5 = self.conv_last(x5, is_pool=False)  #256*256*64
        
        x6 = self.last(x5)                     #256*256*2
        
        
        return x6

In [ ]:
model = Net()

In [ ]:
#model.summary()

In [ ]:
class Dice_coef(tf.keras.metrics.Metric):
    def __init__(self):
        super(Dice_coef, self).__init__()
        self.total =self.add_weight(name='total',dtype=tf.int32,initializer =tf.zeros_initializer())
        self.count =self.add_weight(name='count',dtype=tf.int32,initializer =tf.zeros_initializer())
        self.count2 =self.add_weight(name='count2',dtype=tf.int32,initializer =tf.zeros_initializer())
    def update_state(self,y_true, y_pred,sample_weight=None):   
        y_pred = tf.argmax(y_pred, axis=-1,output_type=tf.int32)
        y_true_f = tf.keras.backend.flatten(y_true) # 将 y_true 拉伸为一维.
        y_pred_f = tf.keras.backend.flatten(y_pred)
    #print(y_pred_f.shape, y_true_f.shape)
        values =tf.keras.backend.sum(y_true_f * y_true_f)
        values2=tf.keras.backend.sum(y_pred_f * y_pred_f)
        intersection =tf.keras.backend.sum(y_true_f * y_pred_f)
        intersection =2*intersection+1
        self.count2.assign_add(intersection)
        self.total.assign_add(values)
        self.count.assign_add(values2)
    def result(self):
        return self.count2/ (self.total + self.count + 1)

In [ ]:
class MeanIoU(tf.keras.metrics.MeanIoU):
    def __call__(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.argmax(y_pred, axis=-1)
        return super().__call__(y_true, y_pred, sample_weight=sample_weight)
learning_rate_fn = tf.keras.optimizers.schedules.PiecewiseConstantDecay([851/4*5,851/4*30,851/4*50],[0.0002,0.00015,0.0001,0.0001*0.5])
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_fn)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
train_iou = MeanIoU(34, name='train_iou')


test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')
test_iou = MeanIoU(34, name='test_iou')
 
test_recall=Recall()
test_precision=Precision()
test_dice =Dice_coef()

In [ ]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)
    train_iou(labels, predictions)

In [ ]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)
    test_iou(labels, predictions)
    test_dice.update_state(labels, predictions)
    test_recall.update_state(labels, predictions)
    test_precision.update_state(labels, predictions)

In [ ]:
train_loss_results = []
train_accuracy_results = []
train_iou_results = []

test_loss_results = []
test_accuracy_results = []
test_iou_result = []
train_loss_results = []
train_accuracy_results = []
train_iou_results = []

test_loss_results = []
test_accuracy_results = []
test_iou_result = []
test_recall_results =[]
test_precision_results =[]
test_dice_results =[]
test_f1_socre_results =[]
test_acc_results =[]


In [ ]:
EPOCHS =70
for epoch in tqdm(range(EPOCHS)):
    sleep(0.01)
    # 在下一个epoch开始时，重置评估指标
    train_loss.reset_states()
    train_accuracy.reset_states()
    train_iou.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    test_iou.reset_states()
    test_dice.reset_states()
    test_recall.reset_states()
    test_precision.reset_states()
    
    

    
    train_loss_results.append(train_loss.result())
    train_accuracy_results.append(train_accuracy.result())
    train_iou_results.append(train_iou.result())

    for test_images, test_labels in dataset_val:
        test_step(test_images, test_labels)
        #test_recall_results.append(sensitivity(labels,images))
        #test_precision_results.append(precision(labels,images))
        #test_dice_results.append(dice_coef(labels[i],images[i]))
        #test_f1_socre_results.append(f1_socre(labels,images))
    test_loss_results.append(test_loss.result())
    test_iou_result.append(test_iou.result())
    test_accuracy_results.append(test_accuracy.result())
    test_dice_results.append(test_dice.result())
    test_recall_results.append(test_recall.result())
    test_precision_results.append(test_precision.result())

    template = 'Epoch {:.3f}, Loss: {:.5f}, Accuracy: {:.3f}, \
                IOU: {:.3f}, Test Loss: {:.5f}, \
                Test Accuracy: {:.3f}, Test IOU: {:.5f},Test Dice: {:.5f},Test Recall: {:.5f},Test Precision: {:.5f},'
    print (template.format(epoch+1,
                           train_loss.result(),
                           train_accuracy.result()*100,
                           train_iou.result(),
                           test_loss.result(),
                           test_accuracy.result()*100,
                           test_iou.result(),
                           test_dice.result(),
                           test_recall.result(),
                           test_precision.result()
                           ))


In [ ]:
a = np.array(test_iou_result)
a

In [ ]:
a  = np.array(test_loss_results)
a

In [ ]:
a  = np.array(train_iou_results)
a

In [ ]:
a  = np.array(train_loss_results)
a

In [ ]:
a  = np.array(test_dice_results)
a

In [ ]:
os.makedirs('./rem')


In [ ]:
../input/lungdatasets3/lung2

In [ ]:
for i in range(20):
    for image,mask in dataset_val.take(i):
        pred_mask = model.predict(image)
        pred_mask = tf.argmax(pred_mask,axis=-1)
        pred_mask = pred_mask[...,tf.newaxis]
        filename1 =str('./rem/img'+str(i)+'.png')
        filename2 =str('./rem/label'+str(i)+'.png')
        filename3 =str('../rem/pre'+str(i)+'.png')
        a = np.array(tf.keras.preprocessing.image.array_to_img(image[0]))
        b = np.array(tf.keras.preprocessing.image.array_to_img(mask[0]))
        c = np.array(tf.keras.preprocessing.image.array_to_img(pred_mask[0]))
        cv2.imwrite(filename1,a)
        cv2.imwrite(filename2,b)
        cv2.imwrite(filename3,c)

In [ ]:
for image,mask in dataset_val.take(127):
    pred_mask = model.predict(image)
    pred_mask = tf.argmax(pred_mask,axis=-1)
    pred_mask = pred_mask[...,tf.newaxis]
num = 1
plt.figure(figsize=(25,25))
for i in range (num):
    plt.subplot(num,6,i*num+1)
    plt.imshow(tf.keras.preprocessing.image.array_to_img(image[i]))
    plt.subplot(num,6,i*num+2)
    plt.imshow(tf.keras.preprocessing.image.array_to_img(mask[i]))
    plt.subplot(num,6,i*num+3)
    plt.imshow(tf.keras.preprocessing.image.array_to_img(pred_mask[i]))

In [ ]:
for image,mask in dataset_val.take(107):
    pred_mask = model.predict(image)
    pred_mask = tf.argmax(pred_mask,axis=-1)
    pred_mask = pred_mask[...,tf.newaxis]
    a = np.array(tf.keras.preprocessing.image.array_to_img(image[0]))
    b = np.array(tf.keras.preprocessing.image.array_to_img(mask[0]))
    c = np.array(tf.keras.preprocessing.image.array_to_img(pred_mask[0]))
    filename1 =str('./img'+str(107)+'.png')
    filename2 =str('./label'+str(107)+'.png')
    filename3 =str('./pre'+str(107)+'.png')
    cv2.imwrite(filename1,a)
    cv2.imwrite(filename2,b)
    cv2.imwrite(filename3,c)

In [ ]:
tf.keras.preprocessing.image.array_to_img(pred_mask[0])

In [ ]:
a= tf.keras.preprocessing.image.array_to_img(pred_mask[i])

In [ ]:
import cv2

In [ ]:
a = np.array(a)

In [ ]:
cv2.imwrite('./2.jpg',a)

In [ ]:
b =0
for i in range(len(test_recall_results)):
    c=i+1
    a =test_recall_results[i]
    b =b+a
    if c %370==0:
        b =b/370
        print(b)
        b=0
b =0
for i in range(len(test_precision_results)):
    c=i+1
    a =test_precision_results[i]
    b =b+a
    if c %369==0:
        b =b/369
        print(b)
        b=0
b =0
for i in range(len(test_f1_socre_results)):
    c=i+1
    a =test_f1_socre_results[i]
    b =b+a
    if c %369==0:
        b =b/369
        print(b)
        b=0
b =0
for i in range(len(test_dice_results)):
    c=i+1
    a =test_dice_results[i]
    b =b+a
    if c %369==0:
        b =b/369
        print(b)
        b=0

In [ ]:
test_dice_results
